In [194]:
import pandas as pd
import numpy as np

In [195]:
recipes = pd.read_csv('datasets/archive/RAW_recipes.csv')
interactions = pd.read_csv('datasets/archive/RAW_interactions.csv')

In [196]:
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [197]:
interactions.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [198]:
recipes.info()
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id  

In [199]:
recipes['ingredients']

0         ['winter squash', 'mexican seasoning', 'mixed ...
1         ['prepared pizza crust', 'sausage patty', 'egg...
2         ['ground beef', 'yellow onions', 'diced tomato...
3         ['spreadable cheese with garlic and herbs', 'n...
4         ['tomato juice', 'apple cider vinegar', 'sugar...
                                ...                        
231632    ['celery', 'onion', 'green sweet pepper', 'gar...
231633    ['paprika', 'salt', 'garlic powder', 'onion po...
231634    ['hard-cooked eggs', 'mayonnaise', 'dijon must...
231635    ['butter', 'eagle brand condensed milk', 'ligh...
231636    ['granulated sugar', 'shortening', 'eggs', 'fl...
Name: ingredients, Length: 231637, dtype: object

In [200]:
recipes['ingredients'][1]

"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']"

In [201]:
recipes['ingredients'][1][0]

'['

In [202]:
import ast

In [203]:
recipes['ingredients'] = recipes['ingredients'].apply(ast.literal_eval)

In [204]:
recipes['ingredients'][1][1]

'sausage patty'

## Content-Based Filtering

In [205]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [206]:
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)
ingredient_matrix = tfidf.fit_transform(recipes['ingredients'].apply(lambda x: ' '.join(x)))

In [207]:
ingredient_matrix

<231637x4158 sparse matrix of type '<class 'numpy.float64'>'
	with 3532612 stored elements in Compressed Sparse Row format>

In [208]:
def find_recipes(input_ingredients, top_n=5):
    input_ingredients_str = ' '.join(input_ingredients)

    input_vector = tfidf.transform([input_ingredients_str])
    
    similarity_scores = cosine_similarity(input_vector, ingredient_matrix).flatten()
    
    similar_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    similar_recipes = recipes.iloc[similar_indices][['name','ingredients','steps']]
    # similar_recipes['similarity_score'] = similarity_scores[similar_indices]
    return similar_recipes


In [209]:
find_recipes("sugar")

,name,ingredients,steps
231636,cookies by design sugar shortbread cookies,"[granulated sugar, shortening, eggs, flour, cr...","['whip sugar and shortening in a large bowl , ..."
77192,easy mix cornbread,"[all-purpose flour, cornmeal, caster sugar, ba...","['preheat oven to 180 degree celcius', 'line 8..."
77218,easy morning pancakes,"[flour, sugar, baking powder, salt, egg, olive...","['in a big bowl , mix sugar , salt , egg and o..."
77217,easy morning french toast bake,"[challah, eggs, heavy cream, orange, zest of, ...","['preheat oven to 400', 'slice the challah bre..."
77216,easy moose rump roast,"[moose rump roast, worcestershire sauce, veget...",['trim off all excess fat and rub roast all ov...


# Collaborative Filtering

In [210]:
interactions = pd.read_csv('datasets/archive/RAW_interactions.csv')

In [211]:
interactions.nunique()

user_id       226570
recipe_id     231637
date            6396
rating             6
review       1125282
dtype: int64

In [212]:
interactions['user_id'].corr(interactions['recipe_id'])

0.10005854007948285

In [213]:
interactions['rating'].value_counts()

rating
5    816364
4    187360
0     60847
3     40855
2     14123
1     12818
Name: count, dtype: int64

In [214]:
interactions = interactions [ interactions['rating'] > 0]

In [215]:
interactions['rating'].value_counts()

rating
5    816364
4    187360
3     40855
2     14123
1     12818
Name: count, dtype: int64

In [216]:
grouped_users = interactions.groupby(['user_id'], as_index=False, sort=False).agg({'recipe_id': 'count'}).reset_index(drop=True)
grouped_users = grouped_users.rename(columns={'recipe_id': 'reviews_count'})
grouped_users = grouped_users.sort_values('reviews_count', ascending=False).iloc[:10000, :]

In [217]:
grouped_recipes = interactions.groupby(['recipe_id'], as_index=False, sort=False).agg({'user_id': 'count'}).reset_index(drop=True)
grouped_recipes = grouped_recipes.rename(columns={'user_id': 'reviews_count'})
grouped_recipes = grouped_recipes.sort_values('reviews_count', ascending=False).iloc[:10000, :]

In [218]:
user_filtered = pd.merge(interactions, grouped_users, on='user_id').drop(['reviews_count'], axis=1)

filtered_interactions = pd.merge(user_filtered, grouped_recipes, on='recipe_id').drop(['reviews_count'], axis=1)

print(filtered_interactions.shape)
filtered_interactions.head()


(280216, 5)


,user_id,recipe_id,date,rating,review
0,56680,79222,2006-11-11,5,"Oh, This was wonderful! Had a soup and salad ..."
1,183565,79222,2006-02-13,5,Wow! My family loves this recipe and it is a ...
2,101823,79222,2006-03-21,5,Excellent chowder. This was the perfect warm-...
3,446143,79222,2008-02-01,4,"Oh, how wonderful! I doubled the crab, and ad..."
4,226989,79222,2008-03-07,4,DH and I enjoyed this. However I used it only ...


In [219]:
filtered_interactions.nunique()

user_id        9970
recipe_id      9973
date           6189
rating            5
review       278857
dtype: int64

In [220]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

user_recipe_matrix = filtered_interactions.pivot(index='user_id', columns='recipe_id', values='rating').fillna(0)

user_similarity = cosine_similarity(user_recipe_matrix)
recipe_similarity = cosine_similarity(user_recipe_matrix.T)

def predict_rating_user_user(user_index, recipe_index, top_n=2):
    similar_users_idx, user_similarities = get_top_similar_users(user_index, top_n)

    ratings = user_recipe_matrix.iloc[similar_users_idx, recipe_index]
    
    weighted_rating = np.dot(user_similarities, ratings) / np.sum(np.abs(user_similarities))
    
    return weighted_rating

def predict_rating_recipe_recipe(user_index, recipe_index, top_n=2):
    similar_recipes_idx, recipe_similarities = get_top_similar_recipes(recipe_index, top_n)
    
    ratings = user_recipe_matrix.iloc[user_index, similar_recipes_idx]
    
    weighted_rating = np.dot(recipe_similarities, ratings) / np.sum(np.abs(recipe_similarities))
    
    return weighted_rating

def get_top_similar_users(user_index, top_n=2):
    similar_users = user_similarity[user_index]
    similar_users_idx = similar_users.argsort()[-top_n-1:-1][::-1] 
    return similar_users_idx, similar_users[similar_users_idx]

def get_top_similar_recipes(recipe_index, top_n=2):
    similar_recipes = recipe_similarity[recipe_index]
    similar_recipes_idx = similar_recipes.argsort()[-top_n-1:-1][::-1]  
    return similar_recipes_idx, similar_recipes[similar_recipes_idx]

predicted_rating_user_user = predict_rating_user_user(0, 2, top_n=2)

predicted_rating_recipe_recipe = predict_rating_recipe_recipe(0, 2, top_n=2)

def suggest_recipes_user_user(user_index, top_n=2):
    predicted_ratings = []
    for recipe_index in range(user_recipe_matrix.shape[1]):
        predicted_rating = predict_rating_user_user(user_index, recipe_index, top_n)
        predicted_ratings.append((recipe_index, predicted_rating))
    
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)

    top_recommended_recipes = predicted_ratings[:top_n]
    return [(user_recipe_matrix.columns[idx], rating) for idx, rating in top_recommended_recipes]


recommended_recipes_user_user = suggest_recipes_user_user(0, top_n=2)
print("\nTop 2 recommended recipes for User using User-User CF:")
for recipe, rating in recommended_recipes_user_user:
    print(f"Recipe {recipe} with predicted rating {rating}")
    
def suggest_recipes_recipe_recipe(user_index, top_n=5):
    predicted_ratings = []
    for recipe_index in range(user_recipe_matrix.shape[1]):
        predicted_rating = predict_rating_recipe_recipe(user_index, recipe_index, top_n)
        predicted_ratings.append((recipe_index, predicted_rating))
    
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    
    top_recommended_recipes = predicted_ratings[:top_n]
    return [(user_recipe_matrix.columns[idx], rating) for idx, rating in top_recommended_recipes]

recommended_recipes_recipe_recipe = suggest_recipes_recipe_recipe(0, top_n=5)
print("\nTop 2 recommended recipes for User 0 using Recipe-Recipe CF:")
for recipe, rating in recommended_recipes_recipe_recipe:
    print(f"Recipe {recipe} with predicted rating {rating}")



Top 2 recommended recipes for User using User-User CF:
Recipe 132916 with predicted rating 5.0
Recipe 123588 with predicted rating 2.7525512860841093

Top 2 recommended recipes for User 0 using Recipe-Recipe CF:
Recipe 110139 with predicted rating 1.8438144115168433
Recipe 145464 with predicted rating 1.4229009557268566
Recipe 87378 with predicted rating 1.1367048644883908
Recipe 103811 with predicted rating 1.0995597287802639
Recipe 175469 with predicted rating 1.0683164088792216


In [221]:
user_recipe_matrix = filtered_interactions.pivot(index='user_id', columns='recipe_id', values='rating').fillna(0)
user_recipe_matrix

recipe_id,49,62,66,142,153,167,192,198,232,246,...,514423,514791,516009,517764,517863,518068,518069,518145,518151,524479
user_id,,,,,,,,,,,,,,,,,,,,,
1533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1535,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001436530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001513060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001524408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

R = user_recipe_matrix.values

svd = TruncatedSVD(n_components=10, random_state=42)  
U = svd.fit_transform(R) 
S = np.diag(svd.singular_values_)  
Vt = svd.components_  

R_hat = np.dot(np.dot(U, S), Vt)

def predict_rating(user_index, recipe_index):
    return R_hat[user_index, recipe_index]

predicted_rating = predict_rating(user_index=0, recipe_index=0)  
print(f"Predicted rating for user on recipe: {predicted_rating}")

from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(U)

recipe_similarity = cosine_similarity(Vt)

def get_top_similar_users(user_index, top_n=3):
    similarities = user_similarity[user_index]
    similar_users_idx = similarities.argsort()[::-1][1:top_n+1]  
    return similar_users_idx, similarities[similar_users_idx]

def get_top_similar_recipes(recipe_index, top_n=3):
    similarities = recipe_similarity[recipe_index]
    similar_recipes_idx = similarities.argsort()[::-1][1:top_n+1] 
    return similar_recipes_idx, similarities[similar_recipes_idx]
    
def predict_user_user(user_index, recipe_index, top_n=3):
    similar_users_idx, user_similarities = get_top_similar_users(user_index, top_n)
    
    ratings = user_recipe_matrix.iloc[similar_users_idx, recipe_index]
    
    weighted_rating = np.dot(user_similarities, ratings) / np.sum(np.abs(user_similarities))
    
    return weighted_rating

predicted_rating_user_user = predict_user_user(user_index=0, recipe_index=0)

def predict_recipe_recipe(user_index, recipe_index, top_n=3):
    similar_recipes_idx, recipe_similarities = get_top_similar_recipes(recipe_index, top_n)
    
    ratings = user_recipe_matrix.iloc[user_index, similar_recipes_idx]
    
    weighted_rating = np.dot(recipe_similarities, ratings) / np.sum(np.abs(recipe_similarities))
    
    return weighted_rating

predicted_rating_recipe_recipe = predict_recipe_recipe(user_index=0, recipe_index=0)
                                                       
def recommend_recipes_user(user_index, top_n=5):
    predicted_ratings = R_hat[user_index]
    
    top_recipes_idx = predicted_ratings.argsort()[::-1][:top_n]
    
    return top_recipes_idx, predicted_ratings[top_recipes_idx]
def recommend_recipes_user(user_index, top_n=3):
    predicted_ratings = R_hat[user_index]
    
    top_recipes_idx = predicted_ratings.argsort()[::-1][:top_n]
    
    return top_recipes_idx, predicted_ratings[top_recipes_idx]

recommended_recipes_user, ratings_user = recommend_recipes_user(user_index=0, top_n=3)
print(f"Top 3 recommended recipes for user (User-User CF): {recommended_recipes_user} with predicted ratings {ratings_user}")

recommended_recipes_user, ratings_user = recommend_recipes_user(user_index=0, top_n=3)
print(f"Top 3 recommended recipes for user (User-User CF): {recommended_recipes_user} with predicted ratings {ratings_user}")
def recommend_users_recipe(recipe_index, top_n=5):
    predicted_ratings = R_hat[:, recipe_index]
    
    # Get indices of the top N users
    top_users_idx = predicted_ratings.argsort()[::-1][:top_n]
    
    return top_users_idx, predicted_ratings[top_users_idx]
recommended_users_recipe, ratings_recipe = recommend_users_recipe(recipe_index=0, top_n=3)
print(f"Top 3 users who would like recipe (Recipe-Recipe CF): {recommended_users_recipe} with predicted ratings {ratings_recipe}")


Predicted rating for user on recipe: 0.2537786495303547
Top 3 recommended recipes for user (User-User CF): [1477 1591 3955] with predicted ratings [38.50108374 31.63255998 31.5133748 ]
Top 3 recommended recipes for user (User-User CF): [1477 1591 3955] with predicted ratings [38.50108374 31.63255998 31.5133748 ]
Top 3 users who would like recipe (Recipe-Recipe CF): [2209 2705 2974] with predicted ratings [555.31440589 106.4899167   96.06663116]


In [224]:
import numpy as np

def clip_rating(rating):
    return np.clip(rating, 0, 5)

def predict_user_user(user_index, recipe_index, top_n=3):
    similar_users_idx, user_similarities = get_top_similar_users(user_index, top_n)
    
    ratings = user_recipe_matrix.iloc[similar_users_idx, recipe_index]
    
    weighted_rating = np.dot(user_similarities, ratings) / np.sum(np.abs(user_similarities))
    
    return clip_rating(weighted_rating)

predicted_rating_user_user = predict_user_user(user_index=0, recipe_index=0)

def predict_recipe_recipe(user_index, recipe_index, top_n=3):
    similar_recipes_idx, recipe_similarities = get_top_similar_recipes(recipe_index, top_n)
    
    ratings = user_recipe_matrix.iloc[user_index, similar_recipes_idx]
    
    weighted_rating = np.dot(recipe_similarities, ratings) / np.sum(np.abs(recipe_similarities))
    
    return clip_rating(weighted_rating)

predicted_rating_recipe_recipe = predict_recipe_recipe(user_index=0, recipe_index=0)

def recommend_recipes_user(user_index, top_n=5):
    predicted_ratings = R_hat[user_index]
    
    top_recipes_idx = predicted_ratings.argsort()[::-1][:top_n]

    top_ratings = np.clip(predicted_ratings[top_recipes_idx], 0, 5)
    
    return top_recipes_idx, top_ratings

recommended_recipes_user, ratings_user = recommend_recipes_user(user_index=0, top_n=5)
print(f"Top 3 recommended recipes for user (User-User CF): {recommended_recipes_user} with predicted ratings {ratings_user}")

def recommend_users_recipe(recipe_index, top_n=5):
    predicted_ratings = R_hat[:, recipe_index]
    
    top_users_idx = predicted_ratings.argsort()[::-1][:top_n]
    
    top_ratings = np.clip(predicted_ratings[top_users_idx], 0, 5)
    
    return top_users_idx, top_ratings

recommended_users_recipe, ratings_recipe = recommend_users_recipe(recipe_index=0, top_n=5)
print(f"Top 3 users who would like recipe 101 (Recipe-Recipe CF): {recommended_users_recipe} with predicted ratings {ratings_recipe}")


Top 3 recommended recipes for user (User-User CF): [1477 1591 3955 1514 1975] with predicted ratings [5. 5. 5. 5. 5.]
Top 3 users who would like recipe 101 (Recipe-Recipe CF): [2209 2705 2974 9108 2436] with predicted ratings [5. 5. 5. 5. 5.]


In [ ]:
user_similarity = cosine_similarity(user_recipe_matrix)
print(user_similarity)
user_similarity.shape

In [ ]:
user_predicted_ratings = np.dot(user_similarity, user_recipe_matrix)
user_predicted_ratings

In [ ]:
user_final_ratings = np.multiply(user_predicted_ratings, user_recipe_matrix)
user_final_ratings.head()

In [ ]:
user_final_ratings = np.clip(user_final_ratings, 0, 5)

In [ ]:
user_final_ratings.iloc[1792].sort_values(ascending = False)[0:20]

In [ ]:
recipe_user_matrix = filtered_interactions.pivot(index='recipe_id', columns='user_id', values='rating').fillna(0)

In [ ]:
recipe_similarity = cosine_similarity(recipe_user_matrix)
print(recipe_similarity)
recipe_similarity.shape

In [ ]:
recipe_predicted_ratings = np.dot(recipe_similarity, recipe_user_matrix)
recipe_predicted_ratings

In [ ]:
recipe_final_ratings = np.multiply(recipe_predicted_ratings, recipe_user_matrix)
recipe_final_ratings.head()

In [ ]:
recipe_final_ratings = np.clip(user_final_ratings, 0, 5)

In [ ]:
recipe_final_ratings.iloc[1535].sort_values(ascending = False)[0:10]

In [ ]:
def recommend_recipes(user_id,  n=5):
    recipe_similarity = cosine_similarity(recipe_user_matrix)
    recipe_predicted_ratings = np.dot(recipe_similarity, recipe_user_matrix)
    recipe_final_ratings = np.multiply(recipe_predicted_ratings, recipe_user_matrix)
    recipe_final_ratings = np.clip(user_final_ratings, 0, 5)
    predictions = recipe_predicted_ratings[:, recipe_final_ratings.index.get_loc(user_id)]
    print(predictions)
    top_recommendations = pd.Series(predictions).sort_values(ascending=False).head(n)
    print(top_recommendations)
    recommended_recipes = recipes[recipes['id'].isin(top_recommendations.index)]
    print(recommended_recipes)

recommend_recipes(1535)

## Model Based Recommendation

In [ ]:
user_recipe_array = user_recipe_matrix.values
user_recipe_array

In [ ]:
from scipy.sparse.linalg import svds


In [ ]:
user_recipe_matrix.shape

In [ ]:
U, sigma, Vt = svds(user_recipe_array, k=50)
sigma = np.diag(sigma)

In [ ]:
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [ ]:
predicted_ratings

In [ ]:
predicted_ratings_df = pd.DataFrame(predicted_ratings, index= user_recipe_matrix.index, columns=user_recipe_matrix.columns)

In [ ]:
predicted_ratings_df.info()

In [ ]:
def recommend_recipes(user_id, predicted_ratings_df, original_matrix, n=5):
    user_ratings = original_matrix.loc[user_id]
    unrated_recipes = user_ratings[user_ratings == 0].index
    predictions = predicted_ratings_df.loc[user_id, unrated_recipes]
    top_recommendations = predictions.sort_values(ascending=False).head(n)
    recommended_recipes = recipes[recipes['id'].isin(top_recommendations.index)]
    recommended_recipes = recommended_recipes.assign(predicted_rating=top_recommendations.values)
    
    return recommended_recipes.sort_values(by="predicted_rating", ascending=False)

In [ ]:
recommend_recipes(1533, predicted_ratings_df, user_recipe_matrix)